## Similar Questions search


The idea is that similar questions will have similar answers. So once we can find similar questions then we can just rank there answers ?

Strategy :

1. Search "topics" in question index.

2. get q_n (some number which can yeild 1000 answers) similar questions

3. Now rank these 1000 using RoBERTa-base


In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re
from collections import defaultdict
import numpy as np
from scipy import spatial
import numpy as np

In [13]:
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('../englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query

def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)
queries = []
#"In this example, the title and the question body of topic with id A.1 is printed."
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
topic_reader = TopicReader(topic_file_path)
dict_q_a = defaultdict(list)

for topic_id in ['A.31','A.78', 'A.101']:
#     topic_id = "A.31"
    title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
    body = topic_reader.get_topic(topic_id).question
    body_pro = re.sub('<[^<]+?>', '', body)
    query = title + '. ' + body_pro
    queries.append(query)
    query = query.lower()
    query = remove_stop(query)
    
#     print(topic_reader.get_topic(topic_id).lst_tags)
    print(topic_id, query)
#     print(body_pro)
    print("=================================")
    body = {
        "size": 500,
        "query": {
               "more_like_this" : {
            "fields" : ["title^3", "body"],
            "like" : query,
            "min_term_freq" : 1,
            "max_query_terms" : 12
        }
#             "multi_match": {
#                             "query": query,
#                             "fields":  ["body","title"],
#                             "type": "cross_fields"
#                         }
#             "match": {
#                 "body": query
#             }
         
        }
    }

    res = es.search(index="arq_ans_ques", body=body)
    for result in res['hits']['hits']:
        dict_q_a[topic_id].append(result['_source']['post_id'])

A.31 doubt implication logical reasoning. evident truth table $p \to q$   $p$ false $q$ true, $p \to q$ true.  instances convince truth value.  example:  $p$: quadrilateral cyclic  $q$: opposite angles supplementary  $p$ false $q$ true, $p \to q$ true? 
A.78 $\sum_{n=1}^{\infty} a_n$ converges absolutely prove $\sum_{n=1}^{\infty} a_n^2$ converges. studying real analysis.i proof proofs exist. but, wrong proof.thanks advance     1.we show satisfies cauchy criterion series  2.consider arbitrary $\epsilon &gt; 0$  3.since $\sum a_n$ converges absolutely  exists s.t forall $n &gt; \geq n$ s.t $|\sum_{k=m+1}^{n} |a_k|| = \sum_{k=m+1}^{n} |a_k| &lt; \sqrt{\epsilon}$  4.we show suffices  5.consider arbitrary $n &gt; \geq n$  $|\sum_{k=m+1}^{n} a_k^2|  =  \sum_{k=m+1}^{n} a_k^2 \leq  (\sum_{k=m+1}^{n} |a_k|)^2 &lt; \sqrt{\epsilon}^2 = \epsilon$  6.so conclude $\sum a_n^2$ converges 
A.101 prove squared equal 2 induction.. asked prove induction $n^2\leq 2^n$, told true $ \forall n\in \mathbb{n}

In [15]:
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

dict_ques = {}

with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            dict_ques[obj['post_id']] = obj
            
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
      
dict_ans = {}
    
with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

dict_aid_post = {}
with jsonlines.open(os.path.join(ans_path)) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            obj['body'] = re.sub('<[^<]+?>', '',  obj['body'])
            dict_aid_post[obj['post_id']] = obj['body']

In [14]:
count = 0

dict_topic_ans = {}

# accepted answers can have a separate list and get higher votes

for topic_id in dict_q_a:
    list_ans_idx = []
    list_other_ans = []
    for qid in tqdm(dict_q_a[topic_id]):
        if dict_ques[qid]['accepted_answer_id'] in dict_ans:
            list_ans_idx.append(dict_ques[qid]['accepted_answer_id'])
        if dict_ques[qid]['answers']:
            for ans in dict_ques[qid]['answers']:
                if ans in dict_ans:
                    list_other_ans.append(ans)
    list_ans_idx.extend(list_other_ans)
    dict_topic_ans[topic_id] = list(set(list_ans_idx))
    print(len(list_ans_idx))

KeyError: 2689399

In [9]:
result_list = defaultdict(list)
for topic_id in dict_topic_ans:
    score = 1000
    for ans in dict_topic_ans[topic_id]:
        result_list[topic_id].append((topic_id, ans, score))
        score-=1

In [6]:
# result_list

In [7]:
# with open('es.500.sim.ques.morelikethis2.dat', 'w') as eval_file:
#     for res in result_list:
#         count = 1
#         for tuples in result_list[res][:1000]:
# #             print(tuples)
#             eval_file.write(tuples[0]+'\t'+ '1\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr'+'\n')
# #             print(tuples[0]+'\t'+ '1\t' + tuples[1] +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr')
#             count+=1

In [7]:
from transformers import pipeline
from transformers import *
import torch 

# del feat_ext
torch.cuda.empty_cache()

feat_ext = pipeline("feature-extraction", model="/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000", tokenizer='roberta-base', device=0) #our finetuned model
# feat_ext = pipeline("feature-extraction", model="shauryr/arqmath-roberta-base-3M", tokenizer='roberta-base', device=0)
# feat_ext = pipeline("feature-extraction", model="shauryr/arqmath-roberta-base", tokenizer='roberta-base', device=0)
# tokenizer =  BertTokenizer.from_pretrained('bert-base-cased')
# feat_ext = pipeline("feature-extraction", model="roberta-base", tokenizer='roberta-base', device=0)
ques_emb = []
for query in queries:
    ques_emb.append(np.mean(feat_ext(query)[0], axis=0))

In [8]:
def Sort_Tuple(tup):  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2])
    return tup

In [10]:
dict_q_idx = {'A.31':0,'A.78':1, 'A.101':2}
result_list = []
count = 0
import math
for qid in ['A.31','A.78', 'A.101']:
    tup_postid_sim = []
    for post_id in tqdm(dict_topic_ans[qid]):
#         print(qid,post_id)
#         ans_emb = np.mean(feat_ext(reduce_str(dict_aid_post[post_id],300))[0], axis=0)
        ans_emb = np.mean(feat_ext(dict_aid_post[post_id])[0], axis=0)
#         ans_emb = feat_ext(dict_aid_post[post_id])[0][0]
#         print(ans_emb)
        result = 1 - spatial.distance.cosine(ques_emb[dict_q_idx[qid]], ans_emb)
        if math.isnan(result):
            count+=1
            print(qid, post_id, count)
#             tup_postid_sim.append((qid,post_id,str(1.0)))
            continue
        tup_postid_sim.append((qid,post_id,result))
    
    print(len(tup_postid_sim))
    print(qid)
    result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:1000])



1042
A.31



650
A.78



1043
A.101


In [11]:
with open('es.mlt.qai.3M.limtfidf.dat', 'w') as eval_file:
    for res in result_list:
        count = 1
        for tuples in res:
#             print(tuples)
            eval_file.write(tuples[0]+'\t'+ '1\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr'+'\n')
#             print(tuples[0]+'\t'+ '1\t' + tuples[1] +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr')
            count+=1